In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

import keras

plt.style.use('fivethirtyeight')

In [ ]:
df=pd.read_csv('./Data/all_stocks_5yr.csv')
df

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

df = df.set_index('date')  # set the date column as the index

df = df.sort_index()


In [ ]:
df = df.drop("Name", axis=1)
df

In [ ]:
#Visualize the closind price history
plt.Figure(figsize=(15,20))
plt.title('close Price History')
plt.plot(df['close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD$', fontsize=18)
plt.show()

In [ ]:
df.shape

In [ ]:
import math

# print(df['close'])
data = df.filter(['close'])
dataset = data.values

training_data_len = math.ceil(len(dataset)*.8)

print(training_data_len)
print(dataset)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data = scaled_data[0:training_data_len,:]
print(training_data_len)
X_train = []
y_train = []

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

    if i<= 60:
        print(X_train)
        print(y_train)

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape

In [ ]:
X_train = np.reshape(X_train, (495172, 60, 1))
X_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(X_train.shape[1],1),))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['MSE','MAE','accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=2, batch_size=100)
history

In [ ]:
test_data = scaled_data[training_data_len-60:,:]
# print(scaled_data)
X_test=[]
y_test=dataset[training_data_len:,:]
for i  in  range(60,len(test_data)):
    X_test.append(test_data[i-60:i,0])
X_test

In [ ]:
X_test=np.array(X_test)

In [ ]:
X_test

In [ ]:
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
predictions=model.predict(X_test)
predictions=scaler.inverse_transform(predictions)
predictions

In [ ]:
rmse=np.sqrt(np.mean(predictions-y_test)**
2)
rmse

In [ ]:
train=data[:training_data_len]
valid=data[training_data_len:]
valid['predictions'] = predictions


plt.Figure(figsize=(20,18))
plt.title('Model')
plt.xlabel('Date')
plt.plot(train['Close'])
plt.plot(valid[['Close','predictions']])
plt.legend(['train','val','predictions'],loc='upper right')
plt.show()

In [ ]:
loss = model.evaluate(X_test, y_test)

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions

In [ ]:
actual_prices = scaler.inverse_transform(np.array(y_test).reshape(-1, 1))
predicted_prices = scaler.inverse_transform(predictions)

In [ ]:
import plotly.graph_objs as go

In [ ]:
df.index

In [ ]:
time_steps=20

In [ ]:
actual_trace = go.Scatter(
    x = df.index[training_data_len+time_steps+1:],
    y = actual_prices.flatten(),
    mode = 'lines',
    name = 'Actual'
)

predicted_trace = go.Scatter(
    x=df.index[training_data_len+time_steps+1:],
    y = predicted_prices.flatten(),
    mode = 'lines',
    name = 'Predicted'
)

# Create layout for the plot
layout = go.Layout(
    title = 'Actual vs. Predicted Apple Stock Prices (LSTM)',
    xaxis = dict(title = 'Date'),
    yaxis = dict(title = 'Price')
)

# Create figure and add traces
fig = go.Figure(data=[actual_trace, predicted_trace], layout=layout)

# Show the plot
fig.show()